# Get Paper Info
These notebook will get the paper info for each speakers in the conference. The attributes are: Number of papers (by year), first paper year, number of citation (by year), country

In [2]:
%run setup.py

### Load the data

In [3]:
# Append all files in Data folder as a datafrom, and add a column for the file name without the extension and folder name
def append_data(folder):
    all_files = glob.glob(folder + "/*.csv")
    df = pd.DataFrame()
    for file in all_files:
        df_temp = pd.read_csv(file, index_col=None)
        df_temp['File name'] = os.path.splitext(os.path.basename(file))[0]
        df = df.append(df_temp, ignore_index=True)
    df['Conference (short)'] = df['File name'].str.split('_').str[0]
    df = df.drop(["Field","File name"], axis=1)
    try:
        df = df.drop(["Key Note Speaker"], axis=1)
    except:
        pass
    return df

df = append_data('Key Note Speakers')
print("Number of rows:", len(df))
print("Number of unique speakers:", df['Full name'].nunique())

df['dblp name'] = df['Full name'].str.replace(' ', '%20')
df.head()

Number of rows: 2044
Number of unique speakers: 1797


,Full name,Year,Sex,Conference (short),dblp name
0,Atsushi Asada,1995,0,ASPDAC,Atsushi%20Asada
1,Jim Meadlock,1995,0,ASPDAC,Jim%20Meadlock
2,John Darringer,1995,0,ASPDAC,John%20Darringer
3,Tatsuo Izawa,1997,0,ASPDAC,Tatsuo%20Izawa
4,Daniel D. Gajski,1997,0,ASPDAC,Daniel%20D.%20Gajski


## dblp.org

In [4]:
df1 = pd.DataFrame(columns = ['Full name','Year','Year Count'])

for Author in df['dblp name']:
    print(Author)
    author_list = []
    paper_list = []
    
    ##### Check if author is on dblp #####
    dblp_URL = "https://dblp.org/search?q=" + Author
    page = requests.get(dblp_URL)
    soup = BeautifulSoup(page.content, "html.parser")
    control_check = soup.find(id="completesearch-authors").find_all("ul",class_ = "result-list")
    if len(control_check) > 0:
        first_author = soup.find(id="completesearch-authors").find_all("ul",class_ = "result-list")[0].find_all("a",href=True)[0]['href']
        print(Author," is on the webpage and the process continues...")
        
        ##### Get info from author #####
        author_page = requests.get(first_author)
        soup_of_speaker = BeautifulSoup(author_page.content, "html.parser")

        Year_list = []

        sections = soup_of_speaker.find(id="publ-section").find_all("div", class_="hide-body")
        for s in range(0,len(sections)): # Nu går vi ind på hver enkelt sektion
            rows_in_sections = sections[s].find_all('ul', class_="publ-list")[0].findAll(True, {"class":['year','entry inproceedings toc','entry article toc','entry incollection toc', 'entry book toc','entry editor toc','entry reference toc']})
            last_row_idx = 0
            
            for row in range(0,len(rows_in_sections)): # Nu går vi ind på hver enkelt række
                #### Append year multipliers
                if rows_in_sections[row].p == None:
                    Year_list.append(rows_in_sections[row].text)
                    paper_list.append(row-last_row_idx-1)
                    last_row_idx = row 
                if row == len(rows_in_sections)-1: # den sidste linje i hver sektion
                    paper_list.append(len(rows_in_sections)-last_row_idx-1)
            
            # remove all -1 values from the list
            paper_list = [x for x in paper_list if x >= 0]
                
        # create a dataframe
        df2 = pd.DataFrame(columns = ['Full name','Year','Year Count'])
        # append full name to the dataframe
        
        df2['Year'] = Year_list
        df2['Year Count'] = paper_list
        df2['Full name'] = Author
        df1 = df1.append(df2, ignore_index=True)
        

Atsushi%20Asada
Atsushi%20Asada  is on the webpage and the process continues...
Jim%20Meadlock
John%20Darringer
John%20Darringer  is on the webpage and the process continues...
Tatsuo%20Izawa
Tatsuo%20Izawa  is on the webpage and the process continues...
Daniel%20D.%20Gajski
Daniel%20D.%20Gajski  is on the webpage and the process continues...
Neil%20Weste
Neil%20Weste  is on the webpage and the process continues...
Biswadip%20Mitra
Biswadip%20Mitra  is on the webpage and the process continues...
Kazuo%20Yano
Kazuo%20Yano  is on the webpage and the process continues...
Aart%20J.%20de%20Geus
Aart%20J.%20de%20Geus  is on the webpage and the process continues...
Martin%20F.H.%20Schuurmans
Martin%20F.H.%20Schuurmans  is on the webpage and the process continues...
Gadi%20Singer
Gadi%20Singer  is on the webpage and the process continues...
Hisashi%20Yamada
Hisashi%20Yamada  is on the webpage and the process continues...
Anantha%20Chandrakasan
Anantha%20Chandrakasan  is on the webpage and the 

KeyboardInterrupt: 

In [23]:
author_list = []
for Author in df['dblp name']:
    print(Author)
    dblp_URL = "https://dblp.org/search?q=" + Author
    page = requests.get(dblp_URL)
    soup = BeautifulSoup(page.content, "html.parser")
    control_check = soup.find(id="completesearch-authors").find_all("ul",class_ = "result-list")
    if len(control_check) > 0:
        first_author = soup.find(id="completesearch-authors").find_all("ul",class_ = "result-list")[0].find_all("a",href=True)[0]['href']

        ##### Get info from author #####
        author_page = requests.get(first_author)
        soup_of_speaker = BeautifulSoup(author_page.content, "html.parser")

        # Title_list = []
        Year_list = []
        final_year_list = []
        final_paper_count = []
        # last_row_idx = 0

        sections = soup_of_speaker.find(id="publ-section").find_all("div", class_="hide-body")
        for s in range(0,len(sections)):
            rows_in_sections = sections[s].find_all('ul', class_="publ-list")[0].findAll(True, {"class":['year','entry inproceedings toc','entry article toc']})
            last_row_idx = 0
            paper_count = []
            for row in range(0,len(rows_in_sections)):

                #### Append year multipliers
                if rows_in_sections[row].p == None:
                    Year_list.append(rows_in_sections[row].text)
                    paper_count.append(row-last_row_idx-1)
                    last_row_idx = row 
                if row == len(rows_in_sections)-1:
                    paper_count.append(len(rows_in_sections)-last_row_idx-1)
                
            final_paper_count.append(paper_count)
        if len(final_paper_count) > 1:
            # Remove the first element
            for i in range(len(paper_count)):
                final_paper_count[i] = [x for x in final_paper_count[i] if x >= 0]
                final_paper_count[i] = final_paper_count[i][1:]
                # remove any negavetive values from the list 
                
                 
                
            # comvert list of lists to one list
            flat_list = [item for sublist in final_paper_count for item in sublist]

            # Multiply the years with the number of papers
            for i in range(0,len(flat_list)):
                for j in range(0,flat_list[i]):
                    final_year_list.append(Year_list[i])
        
        author_list.append(df['Full name']*len(final_year_list))

Atsushi%20Asada


IndexError: list index out of range

In [142]:
# #### Find the URL to the author's page on dblp #####
# dblp_URL = "https://dblp.org/search?q=" + "Victor%20Zue"
# page = requests.get(dblp_URL)
# soup = BeautifulSoup(page.content, "html.parser")
# first_author = soup.find(id="completesearch-authors").find_all("ul",class_ = "result-list")[0].find_all("a",href=True)[0]['href']

# ##### Get info from author #####
# author_page = requests.get(first_author)
# soup_of_speaker = BeautifulSoup(author_page.content, "html.parser")

# Title_list = []
Year_list = []
final_year_list = []
final_paper_count = []
# last_row_idx = 0

sections = soup_of_speaker.find(id="publ-section").find_all("div", class_="hide-body")
for s in range(0,len(sections)):
    rows_in_sections = sections[s].find_all('ul', class_="publ-list")[0].findAll(True, {"class":['year','entry inproceedings toc','entry article toc']})
    last_row_idx = 0
    paper_count = []
    for row in range(0,len(rows_in_sections)):

        #### Append year multipliers
        if rows_in_sections[row].p == None:
            Year_list.append(rows_in_sections[row].text)
            paper_count.append(row-last_row_idx-1)
            last_row_idx = row 
        if row == len(rows_in_sections)-1:
            paper_count.append(len(rows_in_sections)-last_row_idx-1)
        
    final_paper_count.append(paper_count)

# Remove the first element
for i in range(len(liste)):
    final_paper_count[i] = final_paper_count[i][1:]
    
# comvert list of lists to one list
flat_list = [item for sublist in final_paper_count for item in sublist]

# Multiply the years with the number of papers
for i in range(0,len(flat_list)):
    for j in range(0,flat_list[i]):
        final_year_list.append(Year_list[i])


print(final_year_list)    

In [ ]:
        
        # print(rows_in_sections[row].text)
        # print('total rows:',len(rows_in_sections))
        # print('row:',row)
        # print('last_row_idx:',last_row_idx)
        # if row == 0:
        #     year_index += 1
        # else:
        #     # year_index += 1
        #     year_index = row - last_row_idx + 1
        # last_row_idx = row    
        
        # print(year_index)
    
    # Year_list.append((rows_in_sections[row].text)*year_index)
    # print(rows_in_sections[row])    
        
        # print(Year_init)
        # Year_list.append(rows_in_sections[row].text)
    # if rows_in_sections[row].p != None:

# Year_list 

# Find title
# sections[0].find_all('ul', class_="publ-list")[0].findAll(True, {"class":['year','entry inproceedings toc']})[1].find_all('span',class_="title")[0].text

# for s in sections:
#     a = s.find_all("ul", class_="publ_list")
#     print(a)

In [200]:
paper_name = []
paper_year = []
papers = soup_of_speaker.find(id="publ-section").find_all("div", class_="hideable")
for t in range(0,len(papers)): 
    years = papers[t].find_all('li', class_="year")
    #### Append year for each paper
    for y in range(0,len(years)):
        year = years[y].get_text()
        n_papers_in_period = len(papers[t].find_all('cite', class_="data tts-content"))
        paper_year.append(year*n_papers_in_period)
    
    #### Append name of each paper
    for i in range(0, n_papers_in_period):
        paper = papers[t].find_all('cite', class_="data tts-content")[i].find_all('span', class_="title")[0].get_text()
        paper_name.append(paper)

# paper_name

[<ul class="publ-list"><li class="year">2016</li><li class="entry inproceedings toc" id="conf/icalt/ZhangLLZ16" itemscope="" itemtype="http://schema.org/ScholarlyArticle"><link href="https://dblp.org/rdf/schema#Publication" itemprop="additionalType"/><div class="box"><img alt="" src="https://dblp.org/img/n.png" title="Conference and Workshop Papers"/></div><div class="nr" id="c123">[c123]</div><nav class="publ"><ul><li class="drop-down"><div class="head"><a href="https://doi.org/10.1109/ICALT.2016.25"><img alt="" class="icon" itemprop="image" src="https://dblp.org/img/paper.dark.hollow.16x16.png"/></a></div><div class="body"><p><b>view</b></p><ul><li class="ee"><a href="https://doi.org/10.1109/ICALT.2016.25" itemprop="url"><img alt="" class="icon" src="https://dblp.org/img/paper.dark.16x16.png"/>electronic edition via DOI</a></li><li class="ee"><a href="http://doi.ieeecomputersociety.org/10.1109/ICALT.2016.25" itemprop="url"><img alt="" class="icon" src="https://dblp.org/img/paper.dark